In [1]:
from ufal.udpipe import Model, Pipeline, Sentence, OutputFormat
from collections import defaultdict
from bs4 import BeautifulSoup
import numpy as np

In [2]:
import fastText as Fasttext
import gensim
from scipy.spatial.distance import cosine

In [3]:
from work import *
from read import *
from test_embedder import read_data
from deeppavlov.models.embedders.fasttext_embedder import FasttextEmbedder

/cephfs/home/sorokin/env/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/cephfs/home/sorokin/env/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
[nltk_data] Downloading package punkt to /home/sorokin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sorokin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/sorokin/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloa

In [5]:
COUNTS_PATH = "../data/frequencies/Taiga_freq_lemmas.txt"
TRAIN_SAVE_PATH = "paraphraser/parsed_paraphrases_train.xml"
EMBEDDINGS_PATH = "/cephfs/home/sorokin/data/embeddings/ft_native_300_ru_wiki_lenta_lower_case.bin"
CONFIG_PATH = "config/config.json"
TRAIN_SAVE_PATH = "paraphraser/parsed_paraphrases_train.xml"

In [6]:
word_counts = read_counts(COUNTS_PATH, 1, 2)

In [7]:
word_embedder = FasttextEmbedder(EMBEDDINGS_PATH, dim=300)

2018-09-20 21:26:17.382 INFO in 'deeppavlov.models.embedders.fasttext_embedder'['fasttext_embedder'] at line 88: [loading embeddings from `/cephfs/home/sorokin/data/embeddings/ft_native_300_ru_wiki_lenta_lower_case.bin`]


In [17]:
with open(CONFIG_PATH, "r", encoding="utf8") as fin:
    config_params = json.load(fin)
embedder_params = config_params.get("embedder", dict())
if "tag_weights" in embedder_params:
    tag_weights = embedder_params["tag_weights"][1]
    tag_weights = {tuple(key.split("_")): value for key, value in tag_weights.items()}


In [40]:
from importlib import reload, import_module

reload(import_module('embedders'))
from embedders import SVOFreqPosEmbedder, FreqPosEmbedder

In [41]:
embedder = SVOFreqPosEmbedder(word_embedder, word_counts, **embedder_params)
other_embedder = FreqPosEmbedder(word_embedder, word_counts, **embedder_params)

In [9]:
pairs_train, X_train, y_train = read_parsed_paraphrase_file(TRAIN_SAVE_PATH)

In [48]:
sents = X_train[0][:200]
word_sents = [[elem[2] for elem in sent] for sent in sents]
tag_sents = [[elem[3] for elem in sent] for sent in sents]

In [49]:
first = embedder(sents)
second = other_embedder(word_sents, tag_sents)
reshaped_first = np.sum(first.reshape((first.shape[0], 3, -1)), axis=1)
np.max(np.abs(second - reshaped_first))

6.891757964311296e-08

In [29]:
embedder([sent])

array([[ 0.08564041, -0.00959753, -0.05316142,  0.23022451, -0.05575656,
         0.23358977,  0.24826226,  0.00657901, -0.05484641,  0.32550707,
         0.25717932, -0.27336875,  0.3310878 , -0.23046149, -0.37768093,
         0.19008823,  0.11663172, -0.02667265, -0.06820704,  0.1006752 ,
         0.26849952,  0.20673145, -0.3117826 , -0.18229979, -0.04728508,
        -0.35232386, -0.32676911, -0.03725217,  0.38908672,  0.02139609,
         0.20149387,  0.16116548, -0.09741261, -0.0492532 ,  0.17366813,
         0.01652587,  0.4580484 ,  0.02955527, -0.22005905, -0.21757524,
        -0.12745212, -0.12560388,  0.18259843, -0.27164483, -0.01808748,
        -0.15902348, -0.31539962,  0.24049515, -0.16368254, -0.30402339,
         0.04836215, -0.05736595, -0.15155497,  0.02935879, -0.48992974,
         0.41661945, -0.42085975, -0.19684437,  0.25622836,  0.01713147,
         0.20672339,  0.47235724, -0.08730804,  0.05983808,  0.02999398,
        -0.25095594,  0.01836269, -0.03191655, -0.0